In [1]:
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import rnn

(corpus_indices, char_to_idx, idx_to_char,
 vocab_size) = d2l.load_data_jay_lyrics()

In [2]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                nd.zeros(num_hiddens, ctx=ctx))

    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc,
              b_c, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

In [3]:
def init_lstm_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx),
            nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

In [4]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c,
     W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = nd.sigmoid(nd.dot(X, W_xi) + nd.dot(H, W_hi) + b_i)
        F = nd.sigmoid(nd.dot(X, W_xf) + nd.dot(H, W_hf) + b_f)
        O = nd.sigmoid(nd.dot(X, W_xo) + nd.dot(H, W_ho) + b_o)
        C_tilda = nd.tanh(nd.dot(X, W_xc) + nd.dot(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [8]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 16, 64, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [6]:
d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, ctx, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 211.742581, time 0.84 sec
 - 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
 - 不分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
epoch 80, perplexity 66.498685, time 0.82 sec
 - 分开 我想你这你 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要
 - 不分开 我想你这你 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要
epoch 120, perplexity 15.386931, time 0.83 sec
 - 分开 我想带这样坦 我知要这样活 我爱好觉 我不了好生活 我不好好生活 我不好好生活 我不好觉生活 不知
 - 不分开 我想要这样我 不知不觉 我想要这生我 不知不觉 我该了这生我 后知后觉 我该了这生活 我知好好生活
epoch 160, perplexity 3.860912, time 0.84 sec
 - 分开 我已啊 一步两步三四四四望 泪不休 语沉默娘子 娘子好酒 再来一碗热粥 配上几斤的牛肉 我说店小二
 - 不分开 我已到 你我的睛婆  通在看着我多多烦活 难道你手别你 你开到不不不到我 想要你有你你 一样忙人的


In [9]:
lstm_layer = rnn.LSTM(num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 193.061063, time 0.07 sec
 - 分开 我不的 我不的 我不的 我不的 我不不 你不不 你不不 我想不 你不不 你不不 你不不 你不不 我
 - 不分开 我想的 我不的 我不的 我不不 你不不 我想不 你不不 你不不 你不不 你不不 我想不 你不不 你
epoch 80, perplexity 44.179099, time 0.07 sec
 - 分开 我想你这你 我不好好 我不就好 我不就好 我不就好 我不就好 我不就好 我不就好 我不就好 我不就
 - 不分开 我想你这样我 不知你 我不我 我不好 我不就 我不就好 我不就好 我不就好 我不就好 我不就好 我
epoch 120, perplexity 8.212748, time 0.07 sec
 - 分开 我已定这样 我不要这样牵着 不知后觉 你已了离开秋 后后后觉 我该好好生活 后知后觉 我该好好生活
 - 不分开你 我不要这样 我不能这样牵着 不知后觉 你已了一个秋 后后后觉 我该好好生活 后知后觉 我该好好生
epoch 160, perplexity 2.662440, time 0.07 sec
 - 分开 一直螂热热 有辛苦苦 说家怕空 恨句 从小 一切烟 停么有痛 我不懂 从什么 什么我 停么安中 我
 - 不分开直 我知天 你爱我 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在
